# 2022-09-29 • Summary of two-pass conntest

This notebook is a summary of the previous one,
where all its new code has consolidated in the codebase (see github, `pkg/VoltoMapSim/src/`).

## Imports

In [3]:
#

In [4]:
using MyToolbox

In [6]:
using VoltoMapSim

[ Info: Precompiling VoltoMapSim [f713100b-c48c-421a-b480-5fcb4c589a9e]


## Params

In [7]:
p = get_params(
    duration = 10minutes,
    p_conn = 0.04,
    g_EE = 1,
    g_EI = 1,
    g_IE = 4,
    g_II = 4,
    ext_current = Normal(-0.5 * pA/√seconds, 5 * pA/√seconds),
    E_inh = -80 * mV,
    record_v = [1:40; 801:810],
);

## Load STA's

(They're precalculated).

In [51]:
out = cached_STAs(p);

Loading cached output from `C:\Users\tfiers\.phdcache\calc_all_STAs\b9353bdd11d8b8cb.jld2` … done (10.9 s)


In [60]:
(conns, STAs, shuffled_STAs) = out;

In [59]:
Base.summarysize(out) / GB

3.18

In [304]:
# Print info on output of `get_connection_to_test()`
function summarize_conns_to_test(ctt)
    n_post = length(unique(ctt.post))
    println("We test ", nrow(ctt), " putative input connections to ", n_post, " neurons.")
    n(typ) = count(ctt.conntype .== typ)
    println(n(:exc), " of those connections are excitatory, ", n(:inh), " are inhibitory, ",
            "and the remaining ", n(:unconn), " are non-connections.")
end

summarize_conns_to_test(conns)

We test 3906 putative input connections to 50 neurons.
1515 of those connections are excitatory, 391 are inhibitory, and the remaining 2000 are non-connections.


## Test

First peak-to-peak, strict alpha of 0.01 (`tc_ptp`).

Then correlate with found average exc STA, alpha 0.05 (`tc_corr`).

---

In [67]:
testall(f; α) = test_conns(f, conns, STAs, shuffled_STAs; α);

In [69]:
tc_ptp = testall(test_conn__ptp, α = 0.01);

Testing connections: 100%|██████████████████████████████| Time: 0:00:01


In [314]:
function summarize_test_results(tc, typ)
    pm = perfmeasures(tc)
    println(pm.num_pred[typ], " $typ connections found. ", fmt_pct(pm.precision[typ]), " of those are correct.")
    println("The correct $typ detections make up ", fmt_pct(pm.sensitivity[typ]), " of all true $typ connections.")
end

summarize_test_results(tc_ptp, :exc)

845 exc connections found. 93% of those are correct.
The correct exc detections make up 52% of all true exc connections.


In [308]:
avg_ptp_exc_STA = mean(STAs[conn.pre => conn.post] for conn in eachrow(tc_ptp) if conn.predtype == :exc);

In [309]:
tc_corr = testall(test_conn__corr $ (; template = avg_ptp_exc_STA), α = 0.05);

Testing connections: 100%|██████████████████████████████| Time: 0:00:07


In [316]:
function summarize_test_results(tc)
    summarize_test_results(tc, :exc)
    println()
    summarize_test_results(tc, :inh)
    println()
    summarize_test_results(tc, :unconn)
end

summarize_test_results(tc_corr)

1374 exc connections found. 90% of those are correct.
The correct exc detections make up 82% of all true exc connections.

529 inh connections found. 69% of those are correct.
The correct inh detections make up 94% of all true inh connections.

2003 unconn connections found. 85% of those are correct.
The correct unconn detections make up 85% of all true unconn connections.


In [312]:
perftable(tc_corr)

Tested connections: 3906,,,,,,
,,┌───────,Real type,───────┐,,Precision
,,unconn,exc,inh,,
┌,unconn,1704,274,25,,85%
Predicted type,exc,139,1235,0,,90%
└,inh,157,6,366,,69%
,,,,,,
Sensitivity,,85%,82%,94%,,
